In [13]:
import pandas as pd

In [14]:
kaggle_file = '../Datasets/Kaggle/video-game-sales-with-ratings/Video_Games_Sales_as_at_22_Dec_2016.csv'
kaggle = pd.read_csv(kaggle_file)

In [22]:
wiki_file = '../Datasets/Wikipedia/List of Video Game Musicians/all_composer_data.txt'
wiki = pd.read_csv(wiki_file, sep=';')

In [30]:
vgmdb_artist_file = '../Datasets/vgmdb/highest_rated_artists_adjusted.txt'
vgmdb_artist = pd.read_csv(vgmdb_artist_file, sep='<')